In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../code")
import importlib
import numpy as np
import pandas as pd

import minirocket_multivariate_variable as mmv

importlib.reload(mmv)
from sklearn.linear_model import LogisticRegression
from utils import get_cognitive_circles_data, get_cognitive_circles_data_for_classification, prepare_cognitive_circles_data_for_minirocket, medoid_per_class

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Fetching data

In [3]:
df_train_class, y_train, df_test_class, y_test = get_cognitive_circles_data_for_classification('../data/cognitive-circles')
df_train_class.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,DR7865,DR7866,DR7867,DR7868,DR7869,DR7870,DR7871,DR7872,DR7873,DR7874
0,378.950623,390.868329,404.769576,422.604988,443.423940,467.209975,482.904738,498.319701,512.784040,527.204489,...,-117.633838,-123.265863,-131.875027,-140.484191,-148.399901,-154.133713,-159.867525,-165.502344,-166.982869,-168.463395
1,454.875312,465.725686,476.588529,488.401496,500.251870,511.177057,522.039900,532.902743,537.995012,542.438903,...,-390.684958,-371.215367,-341.454023,-293.714498,-235.224907,-163.523044,-7.187791,159.877049,216.619759,276.920364
2,462.274314,442.548628,423.781796,405.987531,390.097257,376.152120,363.248130,357.103491,351.678928,343.234414,...,27.476051,20.187588,12.899125,5.610662,-1.677801,-8.966264,-16.254727,-23.543190,-30.262986,-36.886924
3,621.979426,622.958853,623.000000,620.246883,613.720075,605.987531,597.296135,585.975062,573.407107,565.043017,...,10.431053,13.030254,15.629456,18.228658,20.827859,23.427061,26.026263,28.187702,25.183548,22.179395
4,612.955611,613.000000,613.000000,610.266334,605.554863,599.799002,593.087531,584.598005,576.598005,568.775561,...,43.396504,34.576607,25.756711,16.936815,8.116918,-0.702978,-9.522875,-18.342771,-27.162667,-35.982564


In [4]:
X_train, y_train = prepare_cognitive_circles_data_for_minirocket(df_train_class, y_train)
X_test, y_test = prepare_cognitive_circles_data_for_minirocket(df_test_class, y_test)
X_train.shape

(240, 7, 7874)

## Training MiniROCKET

In [23]:
params = mmv.fit_minirocket_parameters(X_train)
out_train = mmv.transform_prime(X_train, parameters=params)
Phi_train = out_train["phi"]
print("Phi_train:", Phi_train.shape)
out_test = mmv.transform_prime(X_test, parameters=params)
Phi_test = out_test["phi"]

Phi_train: (240, 9996)


## Training a classifier

In [7]:
clf_phi = LogisticRegression(max_iter=2000, solver="lbfgs", multi_class="auto", n_jobs=-1).fit(Phi_train, y_train)
from sklearn.metrics import accuracy_score
print('Accuracy score:', accuracy_score(y_test, clf_phi.predict(Phi_test)))


/home/lgalarra/.local/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_116584_6ff79ceeeb504d348df0118f055e0a34_8287b8e501ef4d668a8c364d65eb4fec for automatic cleanup: unknown resource type folder
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /de

Accuracy score: 0.75


In [11]:
Phi_train.std(axis=1)

array([0.33531576, 0.25649467, 0.33205494, 0.31737602, 0.3356113 ,
       0.31459475, 0.32865927, 0.28780955, 0.34973037, 0.30698985,
       0.32790366, 0.29466298, 0.31286293, 0.29130468, 0.30586714,
       0.29683924, 0.23990344, 0.315601  , 0.31302693, 0.31708828,
       0.31855553, 0.31183   , 0.31809226, 0.32999504, 0.32575056,
       0.26616645, 0.33070344, 0.26798227, 0.3073488 , 0.23096585,
       0.28989047, 0.2941794 , 0.21363376, 0.20384373, 0.22414657,
       0.20408408, 0.27938408, 0.28216255, 0.34213927, 0.28513527,
       0.2675626 , 0.26004076, 0.27723458, 0.2561114 , 0.25305763,
       0.24157056, 0.27757564, 0.28521633, 0.32716915, 0.30775288,
       0.31804195, 0.32121447, 0.33027673, 0.3196955 , 0.22674914,
       0.31094018, 0.246631  , 0.2269703 , 0.2269145 , 0.31086808,
       0.3261347 , 0.32188183, 0.31366217, 0.3128657 , 0.33132744,
       0.33478707, 0.2447153 , 0.27911878, 0.23195735, 0.23071747,
       0.31680614, 0.31611836, 0.3298148 , 0.24229604, 0.24095

In [16]:
def invert_class(y: pd.Series):
    if len(pd.unique(y)) == 2:
        return y.map(lambda x: 1 if x == 0 else 0)
    else:
        ## TODO: Support multi-class
        return y

def get_opposite_medoid_ids(df: pd.DataFrame, y: pd.Series, medoids_per_class: dict):
    y_opposite = invert_class(y)
    opposite_medoids = [medoids_per_class[i] for i in y_opposite.values]
    return opposite_medoids

def _ensure_TC(x):
    x = np.asarray(x)
    if x.ndim == 3:
        assert x.shape[0] == 1
        x = x[0]
    if x.ndim == 2:
        return x.T if x.shape[0] < x.shape[1] else x
    raise ValueError(f"Forma no soportada: {x.shape}")

def _flatten_tc(x_tc):
    return _ensure_TC(x_tc).reshape(-1)
    
medoid_per_class = medoid_per_class(df_train_class, y_train)
opposite_medoid_ids = get_opposite_medoid_ids(df_train_class, y_train, medoid_per_class)

Explain one instance

In [17]:
IDX_TO_EXPLAIN = 0 

In [18]:
from utils import logistic_gradient, medoid_per_class
y_train_pred = clf_phi.predict(Phi_train)

In [19]:
alphas = logistic_gradient(clf_phi, Phi_train[IDX_TO_EXPLAIN])
out_x  = mmv.transform_prime(X_train[IDX_TO_EXPLAIN])
out_x0  = mmv.transform_prime(X_train[opposite_medoid_ids[IDX_TO_EXPLAIN]])

In [20]:
beta = mmv.propagate_luis(alphas, out_x["traces"], X_train[IDX_TO_EXPLAIN], X_train[opposite_medoid_ids[IDX_TO_EXPLAIN]], mode="channel_energy", per_channel=True)
print(beta)

[[ 9.52370418e-09  4.16877815e-08  8.25241975e-08 ... -3.57699452e-08
   9.33769789e-08 -1.50996607e-08]
 [ 1.25612010e-08  1.02351477e-08  8.87017452e-08 ...  2.17658879e-08
   3.72406255e-08  4.56722861e-09]
 [ 1.89885733e-08 -3.70451860e-08  6.88823688e-08 ... -4.05317840e-08
   8.69342431e-08 -3.92913543e-09]
 ...
 [ 1.61661688e-07  9.00331079e-09 -2.12943244e-07 ... -1.65425031e-07
  -5.30073661e-08  3.42318321e-08]
 [ 1.57235012e-07  3.70719634e-08 -2.27086084e-07 ... -1.66300578e-07
  -9.64437940e-08  5.02448670e-08]
 [ 2.01472635e-07  4.94568053e-08 -1.64456584e-07 ... -1.30734224e-07
  -1.41998952e-07  7.50769899e-08]]


In [21]:
beta.shape

(7874, 7)

In [ ]:
class MinirocketClassifier:

    def __init__(self, classifier):
        self.classifier = classifier
        self.minirocket_params = None
        self.traces_obj = None

    def fit(self, X, y):
        self.minirocket_params = mmv.fit_minirocket_parameters(X)
        self.traces_obj = mmv.transform_prime(X, parameters=self.minirocket_params)
        self.classifier.fit(self.traces_obj["phi"], y)

    def minirocket_transform(self, X):
        pass

    def get_minirocket_representation(self):
        ##TODO: Verify that the thing is fit
        return self.traces_obj["phi"]

    def predict(self, X):
        out = mmv.transform_prime(X, parameters=self.minirocket_params)
        return clf_phi.predict(out["phi"])

    def explain_instances(self, X, reference_policy='opposite_class_medoid', reference=None):
        """

        :param X:
        :param reference_policy: opposite_class_medoid | global_medoid | global_centroid | opposite_class_centroid | opposite_predicted_class_medoid | opposite_predicted_class_centroid | farthest_instance | custom
        :return:
        """
        pass

    def explain_instances_from_explainer(self, X, explainerfn=None):
        if explainerfn is None:
            return self.explain_instances(X)
        return explainerfn(X)

def get_minirocket_classifier(X_train, y_train, classifier=LogisticRegression()):
    """

    :param X_train: numpy array of size (n, C, L), n=num_examples, C=num_channels, L=num_samples_per_channel
    :param y_train: variable of size (n,)
    :param classifier: non-fitted sklearn classifier
    :return: MinirocketClassifier
    """
    pass